In [194]:
#importing relevant libraries
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim 
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json

A list of 6 major shopping centers in Oklahoma City

In [57]:
Mall=['Penn Square Mall', 'Quail Springs Mall', 'OKC Outlets', 'Quail Plaza Shopping Center', 'Penn Park Shopping Center',\
      'Midwest City Town Center Plaza']

Creating a dataframe with the names and Coordinates of the shopping centers

In [58]:
Malls = pd.DataFrame(columns = ['Name', 'lat', 'long'], index= range(6)) 

In [59]:
for i, mall in enumerate(Mall):
    
    Address=mall+' ,Oklahoma City, OK'
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(Address)
    latitude = location.latitude
    longitude = location.longitude
    Malls.iloc[i]=[mall, latitude, longitude]
    

In [60]:
Malls

,Name,lat,long
0,Penn Square Mall,35.5252,-97.5449
1,Quail Springs Mall,35.613,-97.5583
2,OKC Outlets,35.4622,-97.6485
3,Quail Plaza Shopping Center,35.5816,-97.566
4,Penn Park Shopping Center,35.3927,-97.5434
5,Midwest City Town Center Plaza,35.4495,-97.3967


Initializing the map around Oklahoma city

In [61]:
address = 'Oklahoma City, OK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 35.4729886, -97.5170536.


Labeling the map with the shopping centers by blue dots

In [196]:
map_OK = folium.Map(location=[latitude, longitude], zoom_start=10)

for Name, lat, lng in zip(Malls['Name'], Malls['lat'], Malls['long']):
    label = '{}'.format(Name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1,
        parse_html=False).add_to(map_OK)  

    
map_OK

<span style='color:Red'> Github does not show the maps, the screenshots are in my report. </span>

The information for my Foursquare account:

In [177]:
CLIENT_ID = '033E3WCFZBFOMPDT044NJPZVIZGPFGPBC2MCYF4CDNY43H4J' 
CLIENT_SECRET = 'DNKJR3QUX3TFMG5W0M3ORY0VNI1VOVFXMAQJNCSARU5CCHO1'
VERSION = '20180605'
LIMIT = 100 

Creating a function to creat a dataframe from the explored venues:

In [178]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
   # print(venues_list)
    
    return(nearby_venues)

Using the function above to create the OK_venues dataframe with all the venues

In [179]:
OK_venues = getNearbyVenues(names=Malls['Name'],
                                   latitudes=Malls['lat'],
                                   longitudes=Malls['long']
                                  )

Penn Square Mall
Quail Springs Mall
OKC Outlets
Quail Plaza Shopping Center
Penn Park Shopping Center
Midwest City Town Center Plaza


In [180]:
OK_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Penn Square Mall,35.525154,-97.544939,The Cheesecake Factory,35.526045,-97.547048,American Restaurant
1,Penn Square Mall,35.525154,-97.544939,AMC Penn Square 10,35.524933,-97.543753,Movie Theater
2,Penn Square Mall,35.525154,-97.544939,Lush,35.525016,-97.545810,Cosmetics Shop
3,Penn Square Mall,35.525154,-97.544939,Lolli and Pops,35.524887,-97.545675,Dessert Shop
4,Penn Square Mall,35.525154,-97.544939,BC Clark Jeweler's,35.525332,-97.545500,Jewelry Store


Separating all restaurant from the main dataframe:

In [181]:
Restaurants=OK_venues[OK_venues['Venue Category'].str.contains('restaurant', case=False)] #['Venue Category'].unique()

In [182]:
Restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Penn Square Mall,35.525154,-97.544939,The Cheesecake Factory,35.526045,-97.547048,American Restaurant
10,Penn Square Mall,35.525154,-97.544939,Whiskey Cake Kitchen & Bar,35.523028,-97.543056,New American Restaurant
19,Penn Square Mall,35.525154,-97.544939,Pepperoni Grill,35.524930,-97.546296,Italian Restaurant
20,Penn Square Mall,35.525154,-97.544939,Chick-fil-A,35.524343,-97.544426,Fast Food Restaurant
21,Penn Square Mall,35.525154,-97.544939,Olive Garden,35.521939,-97.544339,Italian Restaurant


Separating all buffets from the main dataframe:

In [183]:
Buffets=OK_venues[OK_venues['Venue Category'].str.contains('buffet', case=False)]

In [184]:
Buffets.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
264,Penn Park Shopping Center,35.392682,-97.543365,Golden Corral,35.392292,-97.544133,Buffet


Let’s see how many restaurants and buffets we got:

In [185]:
Total_venues=len(OK_venues.index)
Number_of_restaurants=len(Restaurants.index)
Number_of_buffets=len(Buffets.index)
Number_of_positive_venues=len(OK_venues.index)-Number_of_restaurants-Number_of_buffets
print('{},{},{},{}'.format(Total_venues, Number_of_restaurants, Number_of_buffets, Number_of_positive_venues))

293,41,1,251


Total venues are 293  
41 restaurant  
1 buffet

Now lets add all restaurants and buffets to the map.  
Restaurant are red dots and buffets are yellow dots.

In [197]:
for Name, lat, lng in zip(Restaurants['Venue'], Restaurants['Venue Latitude'], Restaurants['Venue Longitude']):
    label = '{}'.format(Name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.8,
        parse_html=False).add_to(map_OK)  

for Name, lat, lng in zip(Buffets['Venue'], Buffets['Venue Latitude'], Buffets['Venue Longitude']):
    label = '{}'.format(Name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.8,
        parse_html=False).add_to(map_OK)  
    
map_OK

<span style='color:Red'> Github does not show the maps, the screenshots are in my report. </span>

Counting the number of restaurants and buffets and other venues to calculate the ratings.  
Every restaurant has -2 points  
Every buffet has -3 points  
Any other venue has +1 point

In [187]:
TOT=OK_venues.groupby('Neighborhood').count()[['Venue']]

In [188]:
RS=Restaurants.groupby('Neighborhood').count()[['Venue']]

In [189]:
BF=Buffets.groupby('Neighborhood').count()[['Venue']]

In [190]:
Final=pd.DataFrame(columns = ['Name', 'Restaurants', 'Buffets', 'Other_venues', 'Rating'])

Create a table with the ratings:

In [191]:
for mall in Mall:
    
    try: N_O_B= BF.loc[mall, 'Venue']
    except: N_O_B=0
        
    N_O_R=RS.loc[mall, 'Venue']
    
    N_O_P=TOT.loc[mall, 'Venue']-N_O_B-RS.loc[mall, 'Venue']
    
    Final=Final.append({'Name': mall,
                       'Restaurants':N_O_R,
                       'Buffets': N_O_B,
                       'Other_venues':N_O_P,
                       'Rating':N_O_P-N_O_R*2-N_O_B*3},
                       ignore_index=True)

In [192]:
Final

,Name,Restaurants,Buffets,Other_venues,Rating
0,Penn Square Mall,8,0,52,36
1,Quail Springs Mall,13,0,80,54
2,OKC Outlets,2,0,52,48
3,Quail Plaza Shopping Center,7,0,15,1
4,Penn Park Shopping Center,9,1,35,14
5,Midwest City Town Center Plaza,2,0,17,13
